# Compute abundance of markers from SALMON remapping output files

#### Aim:
In each output file (corresponding to 1 sampling site):
- compute the RPKM (Reads Per Kilobase Million) for each transcript, by dividing the number of reads remapped against it, by the transcript's length and by the total number of reads present in the sample.

- Additionally: modify the outputs of SALMON by adding a column, which identifies each marker ID with a marker type (eg. RAB GTPase or WASH complex)

In [1]:
# Modules

import numpy as np
import pandas as pd

import os

In [2]:
# Directories where SALMON output files are stored
rep='/home/alexandra/Documents/M2Alexandra/Remapping_files/'
# To automatize the calculation
stations_list=os.listdir(rep) # a list of all the files present in directory 'rep' --> the ouptut files

# File storing clearer names for the reference transcripts and gene families to which they belong
rep_markers='/home/alexandra/Documents/M2Alexandra/Data_test/Remapping/'
df_markers=pd.read_table(rep_markers+'markers_names_vf.tsv')

Names=df_markers['Names']
markers=df_markers['Markers:']


#### Loop on the sampling sites:

In [ ]:
for station in stations_list:

    print(station)
    print("Write total number of reads") # Accessible from another SALMON output (salmon_output.log)
    tot_reads=input()

    if tot_reads!=0:

        file=rep+station+'/quant.sf' # Extract SALMON output file

        df=pd.read_table(file)
        
        # Modify the file by adding columns of protein names and protein families
        df['Name']=Names
        df['Markers']=markers

        df.to_csv(rep+station+'_mapping.tsv',sep='\t')
        
        # Computation of the abundance SCORE
        reads=df['NumReads']
        length=df['Length']

        # Additional information on how many reads were remapped in total
        percentage=np.sum(reads)*100/tot_reads
        print("Percentage of total remapped reads: ", percentage)

        
        abundance=np.zeros(len(df))
        for i in range(len(df)):
            if reads.iloc[i]!=0:
                ratio=reads.iloc[i]/(length.iloc[i]*1e-3) # divide by length of transcript in kilobases
                tot_ratio=ratio/tot_reads # then divide by the total number of reads
                abundance[i]=tot_ratio *1e6 # Multiply by factor 1e6 to have more indicative number

        print("Percentage of transcripts with non-zero abundance:", 100*len(np.where(abundance!=0)[0])/len(abundance))

        with_abundance=df.copy()
        with_abundance['RPKM']=pd.Series(abundance)
        #with_abundance.head(60)
        with_abundance.to_csv(rep+station+'_abundance.tsv',sep='\t')

